In [1]:
IMAGE_DIR = "images"
FOLDERS = ["happy", "sad", "relaxed"]

DEVICE ="mps"

In [2]:
import os
import pandas as pd


## Reading the image data

In [3]:
img_files = []
labels = []
label_code = []

for folder in FOLDERS:
    fname = os.path.join(IMAGE_DIR, folder)
    
    for im in os.listdir(fname):
        impath = os.path.join(fname, im)
        
        img_files.append(impath)
        labels.append(folder)
        label_code.append(FOLDERS.index(folder))

In [4]:
dataset = pd.DataFrame(dict(filename=img_files, label=labels, code=label_code))

dataset

,filename,label,code
0,images/happy/6954267558_0dc33282db_b.jpg,happy,0
1,images/happy/814630827_5f07025901_b.jpg,happy,0
2,images/happy/12526134885_dc4d61d636_b.jpg,happy,0
3,images/happy/14692172321_584a9df5a7_b.jpg,happy,0
4,images/happy/3850905959_71e290209d_b.jpg,happy,0
...,...,...,...
13660,images/relaxed/41502236_bef18fbeea_b.jpg,relaxed,2
13661,images/relaxed/10755758324_627817f7c3_b.jpg,relaxed,2
13662,images/relaxed/41738301434_3dcd69ef8c_b.jpg,relaxed,2
13663,images/relaxed/10461793333_50c543543c_b.jpg,relaxed,2


In [5]:
import torch

In [6]:
torch.manual_seed(0)

torch.backends.mps.is_available()

True

In [7]:
device = torch.device(DEVICE)

In [8]:
from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

In [30]:
class DogDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.augments = [T.RandomHorizontalFlip(1), T.RandomRotation(90), T.AutoAugment()] 
        self.normalize = T.Compose([
            T.ConvertImageDtype(torch.float),
            T.Normalize((.485, .456, .406), (.229, .224, .225))
        ])
        
    def __len__(self):
        return self.dataset.shape[0] * (len(self.augments)+ 1)
    
    def classes(self):
        return self.dataset["code"].unique() # returns the no. of unique labels
    
    def __getitem__(self, idx):
        augment = math.floor(idx/self.dataset.shape[0])
        idx = idx % self.dataset.shape[0]
        
        row = self.dataset.iloc[idx,:]
        
        img_path = row["filename"]
        image = read_image(img_path)
        
        label = row["code"]
        
        if augment > 0 :
            image = self.augments[augment - 1].forward(image)
        
        image = self.normalize(image)
        
        return image, int(label), img_path
        
    
        

In [31]:
data = DogDataset(dataset)

In [32]:
train_size = int(0.8*len(data))
test_size = len(data) - train_size

train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size], generator = torch.Generator().manual_seed(1))


In [33]:
from torch.utils.data import DataLoader #helps load data in batches

BATCH_SIZE = 64

EPOCHS = 5 

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

## Building the Neural Network

In [34]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork,self).__init__()
        
        self.bn = nn.BatchNorm2d(64)
        
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 64 ,2),
            self.bn,
            nn.Conv2d(64, 64 ,2),
            self.bn,
            nn.ReLU(True),
            nn.MaxPool2d(2,2)
            
        )
        
        self.dense = nn.Sequential(
            nn.Linear(64*46*46, 64),
            nn.Linear(64, len(classes))
            
        )
        
    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x,1)
        x = self.dense(x)
            
        return x
        
        

In [35]:
labels = data.classes()
labels

array([0, 1, 2])

In [36]:
model = NeuralNetwork(labels).to(device)
model

NeuralNetwork(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=135424, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
)

## Training the model

In [37]:
loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)



In [38]:
size = len(train.dataset)

In [ ]:
for epoch in range(EPOCHS):
    for batch, (images, labels, img_path) in enumerate(train):
        optimizer.zero_grad()
        
        images = images.to(device)
        pred = model(images)
        
        labels = labels.to(device)
        loss = loss_func(pred, labels)
        
        loss.backward()
        optimizer.step()
    
    loss = loss.item()
    print(f"loss: {loss:>7f} [{epoch}]")


In [ ]:
all_preds = list()
all_labels = list()
all_paths = list()

with torch.no_grad():
    for batch, (images, labels, img_paths) in enumerate(test):
        images = images.to(device)
        outputs = model(images, float())
        
        _, preds = torch.max(outputs.data, 1)
        
        all_labels.append(labels)
        all_preds.append(preds)
        all_paths.append(img_paths)

In [ ]:
import numpy as np

preds = np.concatenate([p.cpu.numpy() for p in all_preds])
labels = np.concatenate([p.cpu.numpy() for p in all_labels])

paths = np.concatenate([p for p in all_paths])

In [ ]:
((preds == labels).sum()) / len(labels)


0.5705268943526005


In [ ]:
predictions = pd.DataFrame(dict(pred=preds, label=labels, path=paths))
predictions["correct"] = (predictions["pred"] = predictions["label"])

In [ ]:
predictions["prediction"] = predictions["pred"].apply(lambda x: FOLDERS[int(x)])

In [ ]:
predictions["actual"] = predictions["label"].apply(lambda x: FOLDERS(int(x)))

In [ ]:
predictions.groupby("prediction").apply(lambda x: x["correct"].sum/x.shape[0])

In [ ]:
disp = predictions.iloc[:25, :].copy()

In [ ]:
def image_formatter(path):
    return f'img src="{path}">'

In [ ]:
disp.style.format({'path': image_formatter})